In [1]:
import pandas as pd
# The requests library simplifies the process of making http requests
import requests

import os

1. Using the code from the lesson as a guide and the REST API from https://python.zgulde.net/api/v1/items as we did in the lesson, create a dataframe named items that has all of the data for items.

In [ ]:
base_url = 'https://api.data.codeup.com'
response = requests.get(base_url)
response.status_code
print('Status Code:', response.status_code)
print('200 means OK')

In [ ]:
response.json()

In [ ]:
response = requests.get(base_url + '/documentation')
print(response.json()['payload'])

In [ ]:
# Setup
# identify the endpoint
endpoint = '/api/v1/items'
# create a variable to hold a list
items = []

In [ ]:
# create the URL
url = base_url + endpoint
# make the request
response = requests.get(url)
# handle the response
data = response.json()


In [ ]:
# identify target data
data['payload']['items']

In [ ]:
# create a dataframe of target data to ensure we have the correct data
pd.DataFrame(data['payload']['items'])

In [ ]:
# add the data to the list
items.extend(data['payload']['items'])

In [ ]:
# check list
items

In [ ]:
# find next endpoint
data['payload']['next_page']

In [ ]:
# create a variable to hold the next page
endpoint = data['payload']['next_page']
# create new url
url = base_url + endpoint
# check url
url

In [ ]:
# make the request
response = requests.get(url)
# handle the response
data = response.json()
# check data
data

In [ ]:
# add items to list
items.extend(data['payload']['items'])

In [ ]:
endpoint = data['payload']['next_page']
url = base_url + endpoint
response = requests.get(url)
data = response.json()
items.extend(data['payload']['items'])

In [ ]:
items

In [ ]:
# turn items into a dataframe
items = pd.DataFrame(items)
items

In [ ]:
def acquire_df(item):
    # identify the base url
    base_line = 'https://api.data.codeup.com'
    # identify the primary endpoint
    api = '/api/v1/'
    # create the url
    url = base_line + api
    response = requests.get(url + item)
    data = response.json()
    # create a .csv file to store the data
    csv_name = (item + '.csv')
    # check if the .csv file exists
    if os.path.isfile(csv_name):
        print('File already exists')
        return pd.read_csv(item+'.csv', index_col=0)
    else:
        # create a variable to hold the list of data
        list = data['payload'][item]
        # check if url still has data to be acquired and add to list
        while data['payload']['next_page'] != None:
            response = requests.get(base_line + data['payload']['next_page'])
            data = response.json()
            list.extend(data['payload'][item])
        # turn list into a dataframe
        df = pd.DataFrame(list)
        # write dataframe to .csv file
        df.to_csv(item+'.csv')
    return df


5. Combine the data from your three separate dataframes into one large dataframe.

In [ ]:
df_items = acquire_df('items')
df_stores = acquire_df('stores')


In [ ]:
df_sales = acquire_df('sales')

In [ ]:

def combine_data(df_sales, df_items, df_stores):
    df_sales.rename(columns={'item': 'item_id'}, inplace=True)
    df_sales_items = df_sales.merge(df_items, on='item_id')
    df_sales_items.rename(columns={'store': 'store_id'}, inplace=True)
    df_sales_items.drop(columns=['item_id'], inplace=True)
    df_trinity = df_sales_items.merge(df_stores, on='store_id')
    df_trinity.drop(columns=['store_id'], inplace=True)
    df_trinity.set_index('sale_id', inplace=True)
    return df_trinity

In [ ]:
df_trinity = combine_data(df_sales, df_items, df_stores)
df_trinity

In [ ]:
# change item to item_id in df_sales
df_sales.rename(columns={'item': 'item_id'}, inplace=True)
df_sales.head()

In [ ]:
df_sales.head()

In [ ]:
# Join df_sales and df_items on item_id
df_sales_items = df_sales.merge(df_items, on='item_id')
df_sales_items.head()

In [ ]:
# Drop columns in df_sales_items that are not needed 'Unnamed: 0_x', 'item_id', 'Unnamed: 0_y'
df_sales_items.drop(columns=['Unnamed: 0_x', 'item_id', 'Unnamed: 0_y'], inplace=True)
df_sales_items.head()

In [ ]:
# change store to store_id in df_sales_items
df_sales_items.rename(columns={'store': 'store_id'}, inplace=True)
df_sales_items.head()

In [ ]:
df_stores.head()

In [ ]:
# drop Unnamed: 0 column in df_stores
df_stores.drop(columns=['Unnamed: 0'], inplace=True)
df_stores.head()

In [ ]:
# Join df_sales and df_items on item_id
df_trinity = df_sales_items.merge(df_stores, on='store_id')
df_trinity

In [ ]:
# drop store_id and sale_id columns from df_trinity
df_trinity.drop(columns=['store_id'], inplace=True)
# index df_trinity by sale_id
df_trinity.set_index('sale_id', inplace=True)
df_trinity.head()

6. Acquire the Open Power Systems Data for Germany, which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017. You can get the data here: https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv


In [ ]:
# turn into a dataframe https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv
opsd_germany_daily = pd.read_csv('https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv', index_col=0)
opsd_germany_daily

In [ ]:
# create a function that acquires opsd_germany_daily and saves as opsd_germany_daily.csv
def acquire_opsd_germany_daily():
    opsd_germany_daily = pd.read_csv('https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv', index_col=0)
    opsd_germany_daily.to_csv('opsd_germany_daily.csv')
    return opsd_germany_daily


In [ ]:
df = acquire_opsd_germany_daily()

7. Make sure all the work that you have done above is reproducible. That is, you should put the code above into separate functions in the acquire.py file and be able to re-run the functions and get the same data.

In [2]:
import timeseries_acquire

In [3]:
df_sales = timeseries_acquire.acquire_df('sales')
df_items = timeseries_acquire.acquire_df('items')
df_stores = timeseries_acquire.acquire_df('stores')

File already exists
File already exists
File already exists


In [5]:
df_trinity = timeseries_acquire.combine_data(df_sales, df_items, df_stores)
df_trinity

,sale_amount,sale_date,item_brand,item_name,item_price,item_upc12,item_upc14,store_address,store_city,store_state,store_zipcode
sale_id,,,,,,,,,,,
1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
2,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
3,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
4,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
5,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
...,...,...,...,...,...,...,...,...,...,...,...
912996,63.0,"Wed, 27 Dec 2017 00:00:00 GMT",Choice,Choice Organic Teas Black Tea Classic Black - ...,5.20,47445919221,47445919221,8503 NW Military Hwy,San Antonio,TX,78231
912997,59.0,"Thu, 28 Dec 2017 00:00:00 GMT",Choice,Choice Organic Teas Black Tea Classic Black - ...,5.20,47445919221,47445919221,8503 NW Military Hwy,San Antonio,TX,78231
912998,74.0,"Fri, 29 Dec 2017 00:00:00 GMT",Choice,Choice Organic Teas Black Tea Classic Black - ...,5.20,47445919221,47445919221,8503 NW Military Hwy,San Antonio,TX,78231


In [6]:
df = timeseries_acquire.acquire_opsd_germany_daily()
df

,Consumption,Wind,Solar,Wind+Solar
Date,,,,
2006-01-01,1069.18400,NaN,NaN,NaN
2006-01-02,1380.52100,NaN,NaN,NaN
2006-01-03,1442.53300,NaN,NaN,NaN
2006-01-04,1457.21700,NaN,NaN,NaN
2006-01-05,1477.13100,NaN,NaN,NaN
...,...,...,...,...
2017-12-27,1263.94091,394.507,16.530,411.037
2017-12-28,1299.86398,506.424,14.162,520.586
2017-12-29,1295.08753,584.277,29.854,614.131
